# Grays Harbor Tides with Sine Forcing

First run the GeoClaw code and then use this notebook to examine the gauge results.

The approach used to model the tide is to apply a source term only offshore, to force the ocean surface level to be some desired tidal motion.  In this example, $\eta_o(t) = \sin(2\pi t / T)$, a pure sine wave with amplitude 1 and period $T = 12\times 3600 = $ 12 hours, as shown by the dashed line in the gauge time series plots below.  

In GeoClaw we add a source term to the mass equation so that it becomes
$$
h_t + (hu)_x + (hv)_y = \eta_o'(t)
$$
everywhere in the ocean sufficiently far offshore. Starting with the ocean at rest, and in the absence of any propagating waves this gives $h(x,y,t) = \eta_o(t)$ in this region, as desired.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *

In [ ]:
from IPython.display import Image
import clawpack.pyclaw.gauges as gauges

In [ ]:
figure(400, figsize=(13,6))
clf()
colors = ['k','b','r']

outdir = '_output'

for k,gaugeno in enumerate([0,1102,1187]):
    gauge = gauges.GaugeSolution(gaugeno, outdir)
    t = gauge.t / 3600.   # convert to hours
    q = gauge.q
    eta = q[3,:]
    plot(t, eta, colors[k], label='Gauge %s' % gaugeno)

    # determine amplification and time shift:
    m2 = int(floor(0.75*len(eta)))
    eta2 = eta[m2:]  # last part of eta signal
    etamax2 = eta2.max()
    etamin2 = eta2.min()
    t2 = t[m2:]
    jtmax = argmax(eta2) 
    tshift = (t2[jtmax] - 39.)*3600.
    
    print('At gauge %i, etamin2 = %.3f, etamax2 = %.3f at tshift = %.1f s' \
            % (gaugeno,etamin2,etamax2,tshift))

# forcing:
tperiod = 12
eta = 1.*sin(2*pi*t/tperiod)
plot(t, eta, 'k--', label='Ocean forcing')

legend(loc='upper right')
xlabel('hours')
ylabel('Surface relative to MTL (m)')
grid(True)
title('Sine wave forcing in ocean and resulting GeoClaw gauge results');

xticks(arange(0,t[-1]+0.1,12))
xlim(0,60)

if 1:
    fname = 'GaugeComparison.png'
    savefig(fname, bbox_inches='tight')
    print('Created %s' % fname)

## Notes:

- Gauge 0 is offshore and shows that GeoClaw does produce the desired sine wave with this forcing.

- Gauges 1102 and 1187 are at the locations of the NOAA tide gauges in Westport and Aberdeen, respectively.  The tide at Westport lags the ocean tide by about 2743 seconds and is amplified relative to the specified ocean tide.  The tide at Aberdeen lags by about 3820 seconds and is further amplified.

- The time lag between high tide at Westport and Aberdeen is about $3820-2743 = 1077$ seconds (18 minutes), which agrees well with observations (need to confirm!).

- At Westport the amplifcation factor is approximate 1.1 (it is greater for the high tide and less for the low tide).

- Given an observed tide at Westport that we want to match, we can shift it by about 2743 seconds and diminish it by a factor 1.1 in order to obtain the $\eta_o(t)$ to use for forcing the ocean.  This is illustrated in the example `geoclaw_tides/grays_kingtide2015`.


The image below shows one frame from the computed solution, showing the location of the gauges and also the portion of the ocean that is forced by the full-strength $\eta_o'(t)$ and the region over which this is tapered linearly in longitude to 0 strength, so that there is no tidal forcing directly at the shore.  Note that the tide in Grays Harbor lags the forced tide in the ocean.

In [ ]:
Image('frame0019fig50.png', width=800)